# Libraries

In [0]:
import numpy as np
import pandas as pd

#Dataset

In [0]:
data = pd.read_csv('data2_19.csv')
data.head()

,"D,X1,X2,X3,X4,X5,X6"
0,"0,3,3,3,4,2,4"
1,"0,3,2,3,5,4,3"
2,"1,5,3,3,3,3,5"
3,"0,5,4,3,3,3,5"
4,"0,5,4,3,3,3,5"


# Data Preprocessing

In [0]:
df = pd.DataFrame( { 'X1':[], 'X2':[], 'X3':[], 'X4':[], 'X5':[], 'X6':[], 'Happy':[] })

len_data = len(data)

for i in range(len_data):
  data_string = data.iloc[i][0]
  data_list = data_string.split(',')
  data_dict = {}
  
  for i in range(1,len(data_list)):
    data_dict['X' + str(i)] = [int(data_list[i])]
    
  data_dict['Happy'] = int(data_list[0])
  
  temp_df = pd.DataFrame(data_dict)
  df = pd.concat([df,temp_df])
  
df.head()  

,X1,X2,X3,X4,X5,X6,Happy
0,3.0,3.0,3.0,4.0,2.0,4.0,0.0
0,3.0,2.0,3.0,5.0,4.0,3.0,0.0
0,5.0,3.0,3.0,3.0,3.0,5.0,1.0
0,5.0,4.0,3.0,3.0,3.0,5.0,0.0
0,5.0,4.0,3.0,3.0,3.0,5.0,0.0


# Model Training

In [0]:
Happy_df = df[df['Happy'] == 1.0]
Happy_df.head()

,X1,X2,X3,X4,X5,X6,Happy
0,5.0,3.0,3.0,3.0,3.0,5.0,1.0
0,5.0,5.0,3.0,5.0,5.0,5.0,1.0
0,5.0,4.0,4.0,4.0,4.0,5.0,1.0
0,5.0,2.0,4.0,5.0,5.0,5.0,1.0
0,3.0,2.0,4.0,3.0,4.0,4.0,1.0


In [0]:
Sad_df = df[df['Happy'] == 0.0]
Sad_df.head()

,X1,X2,X3,X4,X5,X6,Happy
0,3.0,3.0,3.0,4.0,2.0,4.0,0.0
0,3.0,2.0,3.0,5.0,4.0,3.0,0.0
0,5.0,4.0,3.0,3.0,3.0,5.0,0.0
0,5.0,4.0,3.0,3.0,3.0,5.0,0.0
0,3.0,1.0,2.0,2.0,1.0,3.0,0.0


In [0]:
model = pd.DataFrame({'Rating':[1,2,3,4,5]})
model = model.set_index('Rating')

for i in range(1,7):
  happy_temp = []
  sad_temp = []
  
  happy_count = Happy_df['X' + str(i)].value_counts()
  sad_count = Sad_df['X' + str(i)].value_counts()
  
  for j in range(1,6):
    # Laplacian Smoothing
    happy_laplacian_smoothing = ( sum(list(happy_count[happy_count.index == j])) + 1 ) / ( happy_count.sum() + 2 )
    sad_laplacian_smoothing = ( sum(list(sad_count[sad_count.index == j])) + 1 ) / ( sad_count.sum() + 2 )
    
    happy_temp.append(happy_laplacian_smoothing)
    sad_temp.append(sad_laplacian_smoothing)
    
  model['Happy_X' + str(i)] = happy_temp.copy()
  model['Sad_X' + str(i)] = sad_temp.copy()

In [0]:
model

,Happy_X1,Sad_X1,Happy_X2,Sad_X2,Happy_X3,Sad_X3,Happy_X4,Sad_X4,Happy_X5,Sad_X5,Happy_X6,Sad_X6
Rating,,,,,,,,,,,,
1,0.013889,0.032787,0.194444,0.262295,0.041667,0.081967,0.027778,0.016393,0.027778,0.098361,0.013889,0.032787
2,0.013889,0.016393,0.333333,0.245902,0.138889,0.131148,0.027778,0.098361,0.125000,0.163934,0.013889,0.032787
3,0.111111,0.278689,0.319444,0.311475,0.361111,0.573770,0.347222,0.409836,0.166667,0.262295,0.083333,0.278689
4,0.263889,0.426230,0.125000,0.180328,0.347222,0.163934,0.430556,0.360656,0.430556,0.377049,0.402778,0.409836
5,0.638889,0.295082,0.069444,0.049180,0.152778,0.098361,0.208333,0.163934,0.291667,0.147541,0.527778,0.295082


# Prediction

In [0]:
def Predict(model,vector):

  length = len(vector)
  for i in range(length):
    vector[i] = int(vector[i])
    
  #print('Vector: {}'.format(vector))

  happy_prob = len(Happy_df) / len(data)
  #happy_prob = 1
  sad_prob = 1 - happy_prob
  #sad_prob = 1
 
  for i in range(length):
    happy_prob *= model['Happy_X' + str(i+1)][vector[i]]
    sad_prob *= model['Sad_X' + str(i+1)][vector[i]]
  
  print('happy_prob = {} , sad_prob = {}'.format(happy_prob,sad_prob))
  if happy_prob >= sad_prob:
    return 1
  else: return 0
    

# Testing

In [0]:
test_data = pd.read_csv('test2_19.csv')
test_data

,"D,X1,X2,X3,X4,X5,X6"
0,"0,5,1,4,4,4,5"
1,"0,5,2,2,4,4,5"
2,"0,5,3,5,4,5,5"
3,"1,3,4,4,5,1,3"
4,"1,5,1,5,5,5,5"
5,"1,4,3,3,4,4,4"
6,"1,5,5,1,1,5,1"
7,"0,4,4,4,4,1,3"
8,"1,5,2,3,4,4,3"
9,"0,5,3,3,1,3,5"


# Accuracy

In [0]:
length_test = len(test_data)
prediction = []
for i in range(length_test):
  test_data_string = test_data.iloc[i][0]
  test_data_list = test_data_string.split(',')
  pred = Predict(model,test_data_list[1:])
  print('predicted: {} and actual: {} \n'.format(pred,int(test_data_list[0])))
  prediction.append( pred == int(test_data_list[0]) )

accuracy = sum(prediction) / len(prediction)
print('Final Accuracy is : {}'.format(accuracy))
 

happy_prob = 0.0022900565744081833 , sad_prob = 0.00023286209532403973
predicted: 1 and actual: 0 

happy_prob = 0.0015703245081656113 , sad_prob = 0.00017464657149302983
predicted: 1 and actual: 0 

happy_prob = 0.0011213889935327813 , sad_prob = 6.492296462023499e-05
predicted: 1 and actual: 0 

happy_prob = 1.2619971128136777e-06 , sad_prob = 1.6932655381604136e-05
predicted: 0 and actual: 1 

happy_prob = 0.00033028287327052473 , sad_prob = 2.485089554841531e-05
predicted: 1 and actual: 1 

happy_prob = 0.001233359040788407 , sad_prob = 0.0019416404456754665
predicted: 0 and actual: 1 

happy_prob = 1.128786306461123e-07 , sad_prob = 4.314391588266547e-08
predicted: 1 and actual: 1 

happy_prob = 6.194302495393802e-06 , sad_prob = 5.697340516633862e-05
predicted: 0 and actual: 0 

happy_prob = 0.0006446595349311456 , sad_prob = 0.000721629930821894
predicted: 0 and actual: 1 

happy_prob = 9.771634296389545e-05 , sad_prob = 3.060341766610404e-05
predicted: 1 and actual: 0 

happy_p